In [ ]:
import os
import re
import pandas as pd
import parse_raw_to_csv

data_dir= "../../experiments/hpl_scaling"

df = pd.DataFrame()

for f in os.listdir(data_dir):
    if len(re.findall(r"m(\d+).txt", f)) > 0:
        df_tmp = parse_raw_to_csv.parse_file_or_folder(os.path.join(data_dir,f), parse_raw_to_csv.parse_map.values())
        df_tmp["m"] = int(re.findall(r"m(\d+).txt", f)[0])
        df_tmp["type"] = "IEC"
        df_tmp["norm_flops"] = float(df_tmp["lu_gflops"])/(244*5)*100
        df = df.append(df_tmp)

for f in os.listdir(data_dir):
    if len(re.findall(r"m(\d+)_pcie.txt", f)) > 0:
        df_tmp = parse_raw_to_csv.parse_file_or_folder(os.path.join(data_dir,f), parse_raw_to_csv.parse_map.values())
        df_tmp["m"] = int(re.findall(r"m(\d+)_pcie.txt", f)[0])
        df_tmp["type"] = "PCIE"
        df_tmp["norm_flops"] = float(df_tmp["lu_gflops"])/(233.4*5)*100
        df = df.append(df_tmp)

for f in os.listdir(data_dir):
    if len(re.findall(r"m(\d+)_pcie_b8.txt", f)) > 0:
        df_tmp = parse_raw_to_csv.parse_file_or_folder(os.path.join(data_dir,f), parse_raw_to_csv.parse_map.values())
        if df_tmp.empty:
            continue
        df_tmp["m"] = int(re.findall(r"m(\d+)_pcie_b8.txt", f)[0])
        df_tmp["type"] = "PCIE"
        df_tmp["norm_flops"] = float(df_tmp["lu_gflops"])/(280*2)*100
        df = df.append(df_tmp)

data_dir= "../../experiments/hpl_scaling/u280"

for f in os.listdir(data_dir):
    if len(re.findall(r"m(\d+)_pcie.txt", f)) > 0:
        df_tmp = parse_raw_to_csv.parse_file_or_folder(os.path.join(data_dir,f), parse_raw_to_csv.parse_map.values())
        df_tmp["m"] = int(re.findall(r"m(\d+)_pcie.txt", f)[0])
        df_tmp["type"] = "PCIE"
        df_tmp["norm_flops"] = float(df_tmp["lu_gflops"])/(253*2)*100
        df = df.append(df_tmp)

df["benchmark"] = df.index
df = df.apply(pd.to_numeric, errors='ignore')
df = df.reset_index()

df.to_csv("hpl_scaling.csv")
df

In [ ]:
b=512
g=8
f=270e6
f_c = 157e6
t_mm =  (((b ** 2)/(g*f)*3.5 + (b/g) ** 3 / f))
#       load and store block   forward rows and cols    update matrix
t_lu =  (b ** 2)/(g*f)*4 + sum(300/f + (b/g)/f_c + (b * (b - g * int(i/g)))/(g ** 2) / f for i in range(b))

def perf_last(m, torus_width=1, time=False, r=5):
    #LU time at the very beginning of the calculation and at the very end
    lu_total = t_lu + max(m - 2, 0) * t_lu
    def lu_runtime(i):
        return t_lu + max(math.ceil(i/torus_width) - 2,0) * t_lu  #+ max(math.ceil(i/torus_width) - 2,0) * (torus_width * 150*b + 700*b)/f
    def mm_runtime(i):
        remaining_blocks = i/torus_width
        return ((remaining_blocks ** 2) / r) * t_mm
    main_calc_time = sum(max(((2 * (i /torus_width) - 1) / r) * t_mm + lu_runtime(i), mm_runtime(i))  for i in range(1,m * torus_width))
    exe_time = lu_total + main_calc_time
    gflops = (2*((m*torus_width*512) ** 3)/3)/ exe_time * 1.0e-9
    if time:
        return exe_time
    else:
        return gflops

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#Just use set_theme() function of seaborn library for a nice looking appearance.
sns.set_theme()

plot_dfs = {}

plot_dfs["BittWare 520N IEC (block=512)"] = df[(df["size"] <= 80*256) & (df["device"].str.contains("p520_max"))]
plot_dfs["BittWare 520N PCIe (block=256)"] = df[(df["size"] <= 80*256) & (df["device"].str.contains("p520_hpc")) & (df["size"]/df["m"] == 256)]
plot_dfs["BittWare 520N PCIe (block=512)"] = df[(df["size"] <= 80*256) & (df["device"].str.contains("p520_hpc")) & (df["size"]/df["m"] == 512)]
plot_dfs["Xilinx U280 PCIe (block=256)"] = df[(df["size"] <= 80*256) & (df["device"].str.contains("xilinx"))]

fig, hpl = plt.subplots(1)
for k in plot_dfs:
    hpl.scatter(plot_dfs[k]["size"], plot_dfs[k]["norm_flops"], label=k, marker="x")
#hpl.plot(list(range(66)), [perf_last(m) for m in range(66)], label="Model")
#hpl.plot(np.linspace(0,66), [(2* 512 ** 3)/(((b ** 2)/(g*f)*2 + (b/g) ** 3 / f)) * 1.0e-9 * 5 for m in np.linspace(0,66)], label="Model", linestyle="--")
#hpl.plot(np.linspace(0,66), [(2* 512 ** 3)/(((b ** 2)/(g*f)*4 + (b/g) ** 3 / f))* 1.0e-9 * 5 for m in np.linspace(0,66)], label="Matrix Multiplication", linestyle="--")
#hpl.annotate("%.1f GFLOP/s" % ((2* 512 ** 3)/(((b ** 2)/(g*f)*2 + (b/g) ** 3 / f)) * 1.0e-9 * 5), (1,(2* 512 ** 3)/(((b ** 2)/(g*f)*2 + (b/g) ** 3 / f)) * 1.0e-9 * 5 + 20))
#hpl.annotate("%.1f GFLOP/s" % ((2* 512 ** 3)/(((b ** 2)/(g*f)*4 + (b/g) ** 3 / f)) * 1.0e-9 * 5), (1,(2* 512 ** 3)/(((b ** 2)/(g*f)*4 + (b/g) ** 3 / f)) * 1.0e-9 * 5 + 20))
hpl.set_xlabel("Matrix width")
hpl.set_ylabel("norm. GFLOP/s")
hpl.legend()
fig.savefig("hpl_scaling_norm.pdf")

In [ ]:
50000/f

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import parse_raw_to_csv_novalidate
import os
import numpy as np

df = pd.DataFrame()

for i in range(40,56):
    df_tmp = parse_raw_to_csv_novalidate.parse_file_or_folder("../experiments/hpl_scaling/m%d_sq_large.txt" % i, parse_raw_to_csv_novalidate.parse_map.values())
    df_tmp["m"] = i
    df = df.append(df_tmp)

df["benchmark"] = df.index
df = df.apply(pd.to_numeric, errors='ignore')
df = df.reset_index()
df

In [ ]:
fig, hpl = plt.subplots(1)
hpl.plot(df["m"], df["lu_gflops"])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import parse_raw_to_csv_novalidate
import os
import numpy as np

df = pd.DataFrame()

for i in [20 + t*5 for t in range(1,11)]:
    df_tmp = parse_raw_to_csv_novalidate.parse_file_or_folder("../experiments/hpl_scaling/m%d_sq.txt" % i, parse_raw_to_csv_novalidate.parse_map.values())
    df_tmp["m"] = i
    df = df.append(df_tmp)

df["benchmark"] = df.index
df = df.apply(pd.to_numeric, errors='ignore')
df = df.reset_index()
df

In [ ]:
fig, hpl = plt.subplots(1)
hpl.plot(df["m"], df["lu_gflops"])
hpl.plot(range(71), [perf(m) for m in range(71)])
hpl.set_ylim(0,1200)